# Replica project

### Plotting model results


In [23]:
# loading the metadata
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import sys
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from unidecode import unidecode

sys.path.insert(0, "../model/")
from utils import *

sys.path.insert(0, "../web_annotation/")
from utils_clusters import *
from metrics_clusters import * 


def remove_non_ascii(text):
    return unidecode(text)

data_dir = '/scratch/students/schaerf/'
data_dir = '../data/'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Compatibility for Pixplot

In [35]:
subfolder_dir = '01-06-2022'
embeds_file = subfolder_dir + '/resnext-101_'+subfolder_dir+'.npy'
positives = update_morph(data_dir, '')
data = pd.read_csv(data_dir + 'data_sample.csv')
embeds = np.load(data_dir + embeds_file, allow_pickle=True) 
    
with open(data_dir + 'uid2path.pkl', 'rb') as infile:
    uid2path = pickle.load(infile)
data.columns

Index(['uid', 'uid_connection', 'AuthorOriginal', 'Description', 'Author',
       'AuthorBirth', 'path', 'BeginDate', 'ImageURL', 'City', 'Country',
       'AuthorBirthCity', 'AuthorDeathCity', 'Drawer', 'Type', 'img1', 'img2',
       'type', 'annotated', 'index', 'cluster_file', 'cluster', 'set'],
      dtype='object')

In [46]:
uid2cluster = dict(((cl,positives[positives['uid'] == cl]['cluster'].values[0]) if positives[positives['uid'] == cl].shape[0] > 0 else (cl, -1) for cl in data.uid))

In [48]:
data['cluster'] = data['uid'].apply(lambda x: uid2cluster[x])

In [49]:
data['path'] = data['uid'].apply(lambda x: uid2path[x])
data['IIIF image'] = data['ImageURL'].apply(lambda x: get_drawer(x, x))
data['filename'] = data['path'].str.split('/').apply(lambda x: x[-1])

data['IIIF image'].values[:30]

array(['http://www.wga.hu/art/g/garnie/diana_ny.jpg',
       'http://www.wga.hu/art/g/garnie/diana_ny.jpg',
       'http://www.wga.hu/art/g/garnie/diana_ny.jpg',
       'http://www.wga.hu/art/g/garnie/diana_ny.jpg',
       'http://www.wga.hu/art/g/garnie/diana_ny.jpg',
       'http://www.wga.hu/art/g/garnie/diana_ny.jpg',
       'http://www.wga.hu/art/g/garnie/diana_ny.jpg',
       'http://www.wga.hu/art/g/garnie/diana_ny.jpg',
       'http://www.wga.hu/art/g/garnie/diana_ny.jpg',
       'http://www.wga.hu/art/g/garnie/diana_ny.jpg',
       'http://www.wga.hu/art/g/garnie/diana_ny.jpg',
       'http://www.wga.hu/art/g/garnie/diana_ny.jpg',
       'http://www.wga.hu/art/g/garnie/diana_ny.jpg',
       'http://www.wga.hu/art/g/garnie/diana_ny.jpg',
       'http://www.wga.hu/art/g/garnie/diana_ny.jpg',
       'http://www.wga.hu/art/g/garnie/diana_ny.jpg',
       'http://www.wga.hu/art/g/garnie/diana_ny.jpg',
       'http://www.wga.hu/art/g/garnie/diana_ny.jpg',
       'https://dhlabsrv4.ep

In [50]:
manifest = data[data['IIIF image'].str.contains('iiif')]
print(manifest.shape)

lines = list(manifest['IIIF image'])
with open(data_dir + 'manifest.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(lines))

(10167, 25)


In [51]:
cat2name = {cl:group['Description'].values[0].replace('S.', 'San').split('.')[0] for cl, group in manifest.fillna(-1).groupby('cluster')}
cat2name[-1.0] = 'Outlier'
cat2name

{-1: 'Outlier',
 1: 'Sacra Famiglia con San Giovannino',
 3: 'Nevicata',
 9: 'La Sacra Famiglia',
 10: 'La Deposizione',
 17: 'Roma antica',
 20: 'Venere e Adone',
 24: 'Mosè salvato dalle acque',
 25: 'San Girolamo',
 29: 'Ganimede rapito al cielo',
 33: 'La toilette di Venere',
 34: "Ratto d'Europa",
 38: 'adonna col Bambino',
 48: 'Deposizione nel sepolcro',
 49: 'La Maddalena penitente',
 50: 'Maddalena',
 51: 'Il Martirio di SanPietro da Verona',
 52: 'Tarquinio e Lucrezia',
 54: 'Venere e Adone',
 63: 'Diana e Atteone',
 72: 'Venere dormiente',
 75: 'Battaglia di Anghiari',
 78: 'Venere, Bacco e Cerere',
 81: 'Giuditta con la testa di Oloferne',
 82: 'La dama del cagnolino',
 84: 'Marte e Venere',
 92: 'Venere e Cupido',
 93: 'LA Danae',
 94: 'C6)',
 95: 'San Giovanni Battista',
 97: 'Salomè',
 100: 'Ninfa e pastore 50,7 x 60,50 cm',
 101: 'Madonna col Bambino e San Giovannino',
 103: 'Sacra Conversazione',
 105: 'Madonna con il Bambino e i Santi Girolamo e Dorotea',
 108: 'Madon

In [54]:
with open(data_dir + subfolder_dir + '/map2pos.pkl', 'rb') as infile:
        map2pos = pickle.load(infile)

In [57]:
manifest['filename'] = manifest['path'].str.split('/').apply(lambda x: x[-1])
manifest['category'] = manifest['cluster'].fillna(-1).apply(lambda x: cat2name[x])
manifest['tags'] = np.nan
manifest['description'] = manifest['Author'] + ': ' + manifest['Description'].fillna('').apply(lambda x: remove_non_ascii(str(x)))
manifest['permalink'] = manifest['IIIF image']
manifest['year'] = manifest['BeginDate']
manifest['x'] = manifest['uid'].apply(lambda x: catch(x, map2pos)[0])
manifest['y'] = manifest['uid'].apply(lambda x: catch(x, map2pos)[1])

manifest['category'].value_counts()

C:\Users\ludov\AppData\Local\Temp/ipykernel_26504/3104953125.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  manifest['filename'] = manifest['path'].str.split('/').apply(lambda x: x[-1])
C:\Users\ludov\AppData\Local\Temp/ipykernel_26504/3104953125.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  manifest['category'] = manifest['cluster'].fillna(-1).apply(lambda x: cat2name[x])
C:\Users\ludov\AppData\Local\Temp/ipykernel_26504/3104953125.py:3: SettingWithCopyWarning: 
A value is trying to be set on a 

Outlier                            7003
Madonna con Bambino                 268
Contemplazione della Morte           85
Maddalena                            66
Madonna col Bambino                  65
                                   ... 
L 'alchimista                         1
Campo Sant 'Angelo                    1
Ninfa e pastore 50,7 x 60,50 cm       1
Ritratto di Giulio II                 1
Rebecca al pozzo                      1
Name: category, Length: 416, dtype: int64

In [58]:
manifest2 = manifest[['filename', 'category', 'tags', 'description', 'permalink', 'year', 'x', 'y']]
manifest2.head()

,filename,category,tags,description,permalink,year,x,y
18,112B_571.jpg,Outlier,NaN,WEENIX Jan-Baptiste: Venditrice di erbaggi.,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,1639.0,-40.796360,-1.354221
20,29B_659.jpg,Outlier,NaN,ALLORI Cristoforo: Uomo inginocchiato. Carbonc...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,1595.0,7.993937,-21.711922
21,147A_683.jpg,Outlier,NaN,SEC XVI: Armadio in noce intagliato.,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,1500.0,44.901394,34.435062
23,2A_268.jpg,Outlier,NaN,CAMPAGNOLA Domenico: L 'Adorazione dei lagi. P...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,1518.0,5.044211,38.256744
24,14A_300.jpg,Outlier,NaN,BURGKMAIR Hans (?): Carro con musici. Penna lu...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,1491.0,13.865476,34.637856


In [8]:
manifest2.to_csv(data_dir + 'metadata.csv', encoding='utf8')

In [9]:
uid2name = {uid:name for uid, name in zip(data['uid'], data['filename'])}

embeds[:, 0] = [catch(x, uid2name) for x in embeds[:, 0]]
embeds


array([['diana_ny.jpg',
        array([0.02669414, 0.01063903, 0.01577589, ..., 0.02917384, 0.01784928,
       0.02706564], dtype=float32)],
       ['112B_571.jpg',
        array([0.01176713, 0.00649136, 0.00810071, ..., 0.0160761 , 0.01098835,
       0.02134623], dtype=float32)],
       ['berenice.jpg',
        array([0.04015105, 0.0246291 , 0.01989905, ..., 0.01684402, 0.01566825,
       0.00978398], dtype=float32)],
       ...,
       ['cleopatr.jpg',
        array([0.03623779, 0.00738333, 0.02136839, ..., 0.04073193, 0.0239862 ,
       0.01873499], dtype=float32)],
       ['90A_173.jpg',
        array([0.00879891, 0.00309545, 0.0331305 , ..., 0.00737908, 0.009536  ,
       0.00992465], dtype=float32)],
       ['63C_44.jpg',
        array([0.0348242 , 0.0098505 , 0.03602742, ..., 0.01236406, 0.01548924,
       0.0203557 ], dtype=float32)]], dtype=object)

In [10]:
iiif2vec = {}

for emb in embeds:
    iiif2vec[emb[0]] = emb[1]

list(iiif2vec.items())[0]

('diana_ny.jpg',
 array([0.02669414, 0.01063903, 0.01577589, ..., 0.02917384, 0.01784928,
        0.02706564], dtype=float32))

In [11]:
with open(data_dir + 'iiif2vec.pkl', 'wb') as outfile:
    pickle.dump(iiif2vec, outfile)

In [ ]:
def get_inception_vectors(**kwargs):
  '''Create and return Inception vector representation of Image() instances'''
  print(timestamp(), 'Creating Inception vectors for {} images'.format(len(kwargs['image_paths'])))
  vector_dir = os.path.join(kwargs['out_dir'], 'image-vectors', 'inception')
  if not os.path.exists(vector_dir): os.makedirs(vector_dir)
  #base = InceptionV3(include_top=True, weights='imagenet',)
  #model = Model(inputs=base.input, outputs=base.get_layer('avg_pool').output)
  with open(data_dir + 'iiif2vec.pkl', 'rb') as infile:
    iiif2vec = pickle.load(infile)
  print(timestamp(), 'Creating image array')
  vecs = []
  with tqdm(total=len(kwargs['image_paths'])) as progress_bar:
    for idx, i in enumerate(stream_images(**kwargs)):
      filename = clean_filename(i.path)

      vector_path = os.path.join(vector_dir, clean_filename(i.path) + '.npy')
      #if os.path.exists(vector_path) and kwargs['use_cache']:
      #  vec = np.load(vector_path)
      #else:
      #  im = preprocess_input( img_to_array( i.original.resize((299,299)) ) )
      #vec = model.predict(np.expand_dims(im, 0)).squeeze()
      vec = iiif2vec[filename]
      np.save(vector_path, vec)
      vecs.append(vec)
      progress_bar.update(1)
  return np.array(vecs)

In [ ]:
def get_cluster_model(**kwargs):
  '''Return a model with .fit() method that can be used to cluster input vectors'''
  if cluster_method == 'hdbscan':
    config = {
      'core_dist_n_jobs': multiprocessing.cpu_count(),
      'min_cluster_size': kwargs['min_cluster_size'],
      'cluster_selection_epsilon': 0.01,
      'min_samples': 1,
      'approx_min_span_tree': False,
    }
    return HDBSCAN(**config)
  elif cluster_method == 'optics':
    print('clustering with optics')
    config = {
      'core_dist_n_jobs': multiprocessing.cpu_count(),
      'min_samples': 2, #kwargs['min_cluster_size'],
      'max_eps': 0.13,
       #'min_cluster_size': 2,
      #'approx_min_span_tree': False,
      'metric':'cosine',
    }
    return OPTICS(**config)
  else:
    return KMeans(n_clusters=kwargs['n_clusters'], random_state=kwargs['seed'])
